# Toronto Neighborhood Clustering

#### Part 1

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.
1. Start by creating a new Notebook for this assignment.
2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe
3. Create a dataframe with the following requirements:
•	The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
•	Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
•	More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
•	If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
•	Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
•	In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
4. Submit a link to your Notebook on your Github repository. (10 marks)


##### Import neccesary libraries

In [1]:
import numpy as np
import pandas as pd
import requests

##### Get the Toronto postal URL. Then read into tables

In [2]:
url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables =pd.read_html(url, header=0)

##### Only use the first table and read in as a dataframe

In [3]:
df=tables[0]
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


##### Drop rows where Borogh is Not assigned

In [4]:
df=df[df['Borough'] != 'Not assigned']
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


##### Reset index

In [5]:
df=df.reset_index(drop=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


##### find Neighbourhood where cell value is 'Not assigned'

In [6]:
df[df['Neighbourhood']=='Not assigned']

,Postcode,Borough,Neighbourhood
6,M7A,Queen's Park,Not assigned


##### assign Borough "Queen's Park" to Neighbourhood cell. Verify it's correctly assigned

In [7]:
df.iloc[6,2]='Queen\'s Park'
df[6:7]

,Postcode,Borough,Neighbourhood
6,M7A,Queen's Park,Queen's Park


##### groupby Postcode and aggregate Neighbourhood into same cell with ', ' separator

In [22]:
df=df.groupby(['Postcode','Borough']).agg({'Neighbourhood': lambda x: ', '.join(x)})
df.head()

,,Neighbourhood
Postcode,Borough,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


##### reset index

In [8]:
df = df.reset_index()
df.head(10)

,index,Postcode,Borough,Neighbourhood
0,0,M3A,North York,Parkwoods
1,1,M4A,North York,Victoria Village
2,2,M5A,Downtown Toronto,Harbourfront
3,3,M5A,Downtown Toronto,Regent Park
4,4,M6A,North York,Lawrence Heights
5,5,M6A,North York,Lawrence Manor
6,6,M7A,Queen's Park,Queen's Park
7,7,M9A,Etobicoke,Islington Avenue
8,8,M1B,Scarborough,Rouge
9,9,M1B,Scarborough,Malvern


In [9]:
df.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)

##### print number of rows value of the dataframe

In [10]:
print('The number of rows is: ', df.shape[0])

The number of rows is:  212


#### Part2

Now that you have built a dataframe of the postal code of each neighborhood along with 
the borough name and neighborhood name, in order to utilize the Foursquare location data,
we need to get the latitude and the longitude coordinates of each neighborhood.

##### get postcode and corresponding lat&lng coordinates csv file

In [11]:
postcode_csv_url ='http://cocl.us/Geospatial_data'
pcode = pd.read_csv(postcode_csv_url)
pcode.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### Merge two dataframes

In [12]:
dfmerge = df.merge(pcode, how='inner', left_on='Postcode', right_on='Postal Code')
dfmerge.head()

,index,Postcode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,2,M5A,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636
3,3,M5A,Downtown Toronto,Regent Park,M5A,43.654260,-79.360636
4,4,M6A,North York,Lawrence Heights,M6A,43.718518,-79.464763


##### drop a duplicate postal code column

In [13]:
dfmerge=dfmerge.drop('Postal Code', axis=1)
dfmerge.head()

,index,Postcode,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,4,M6A,North York,Lawrence Heights,43.718518,-79.464763
